## ユーザー登録API

In [ ]:
import requests

from lib.api_config import DOMAIN
from lib.utils import random_string

json = {
    "email": f"{random_string(10)}@example.com",
    "password": "password",
}
response = requests.post(f"http://{DOMAIN}/users", json=json)

print(response.status_code)
print(response.json())

In [ ]:
!EMAIL=`head -c 32 /dev/urandom | base64 | tr -dc 'a-zA-Z0-9' | head -c 16`; \
jq -n "{email: \"$EMAIL@example.com\", password: \"password\"}" | curl -v --json @- "http://$DOMAIN/users"

## ユーザー認証API

In [ ]:
import requests

from lib.api_config import DOMAIN
from lib.utils import random_string

email = f"{random_string(10)}@example.com"
password = "password"

register_json = {
    "email": email,
    "password": password,
}
register_response = requests.post(f"http://{DOMAIN}/users", json=register_json)
print("Register status:", register_response.status_code)

auth_json = {
    "email": email,
    "password": password,
}
auth_response = requests.post(f"http://{DOMAIN}/auth", json=auth_json)

print("Auth status:", auth_response.status_code)
print("Response:", auth_response.json())

In [ ]:
!EMAIL=`head -c 32 /dev/urandom | base64 | tr -dc 'a-zA-Z0-9' | head -c 16`; \
jq -n "{email: \"$EMAIL@example.com\", password: \"password\"}" | curl -v --json @- "http://$DOMAIN/users" && \
jq -n "{email: \"$EMAIL@example.com\", password: \"password\"}" | curl -v --json @- "http://$DOMAIN/auth"

## タスク作成API

In [ ]:
import datetime
import requests

from lib.api_config import DOMAIN
from lib.utils import random_string

email = f"{random_string(10)}@example.com"
password = "password"

register_json = {
    "email": email,
    "password": password,
}
register_response = requests.post(f"http://{DOMAIN}/users", json=register_json)
print("Register status:", register_response.status_code)

auth_json = {
    "email": email,
    "password": password,
}
auth_response = requests.post(f"http://{DOMAIN}/auth", json=auth_json)
token = auth_response.json()["token"]
print("Auth status:", auth_response.status_code)

deadline = datetime.datetime.now() + datetime.timedelta(days=1)
task_json = {
    "name": "task1",
    "description": "description1",
    "deadline": deadline.strftime("%Y-%m-%dT%H:%M:%SZ"),
    "completed": False
}

task_response = requests.post(
    f"http://{DOMAIN}/tasks",
    headers={
        "Authorization": f"Bearer {token}",
    },
    json=task_json
)

print("Task creation status:", task_response.status_code)
print("Response:", task_response.json())

In [ ]:
!EMAIL=`head -c 32 /dev/urandom | base64 | tr -dc 'a-zA-Z0-9' | head -c 16`; \
jq -n "{email: \"$EMAIL@example.com\", password: \"password\"}" | curl -v --json @- "http://$DOMAIN/users" && \
TOKEN=`jq -n "{email: \"$EMAIL@example.com\", password: \"password\"}" | curl -v --json @- "http://$DOMAIN/auth" | jq -r .token` && \
DATE=`date -u +"%Y-%m-%dT%H:%M:%SZ"`; \
jq -n "{name: \"task1\", description: \"description1\", deadline:\"$DATE\", completed: false}" | \
curl -v -H "Authorization: Bearer $TOKEN" --json @- "http://$DOMAIN/tasks"